In [1]:
from affpyramid import AffPyramid
import numpy as np
from skimage import color
import tifffile as tiff

In [ ]:
image_id = 3760001747
aff_path = '/ACdata/Users/tmp/smi32/' + str(image_id) + '/' + str(image_id) + '.aff'
out_ts_array = '/ACdata/Users/wanqing/data/human_smi32/' + str(image_id) + '.zarr'

In [3]:
aff = AffPyramid(aff_path)

aff.num_tiers   # number of pyramid tiers; the higher tier, the higher resolution

9

In [4]:
# returns a numpy array containing all tiles from a tier stitched together
tier = aff.num_tiers - 2
rgb_image = aff.get_tier(tier)

In [5]:
# convert rgb image to 8bit grayscale
grayscale_array = color.rgb2gray(rgb_image)

# Convert to 8-bit grayscale
grayscale_8bit = (grayscale_array * 255).astype(np.uint8)

grayscale_8bit


array([[28, 28, 28, ..., 28, 28, 28],
       [28, 28, 28, ..., 28, 28, 28],
       [28, 28, 28, ..., 28, 28, 28],
       ...,
       [28, 28, 28, ..., 28, 28, 28],
       [28, 28, 28, ..., 28, 28, 28],
       [28, 28, 28, ..., 28, 28, 28]], dtype=uint8)

In [6]:
import tensorstore as ts

def create_EmptyTensor(filepath, shape, driver = 'zarr3', dtype = 'uint16', fill_value = -np.inf, chunk_shape = [64, 64, 64]):
    out_arr = ts.open({
     'driver': driver,
     'kvstore': 'file://' + str(filepath),
     },
     dtype=dtype,
     fill_value=fill_value,
     chunk_layout=ts.ChunkLayout(chunk_shape=chunk_shape),
     create=True,
     shape=list(shape)).result()
    return out_arr

In [7]:
grayscale_8bit.shape

(32495, 23724)

In [8]:
out_ts_arr = create_EmptyTensor(out_ts_array, grayscale_8bit.shape, dtype = 'uint8', driver = 'zarr3', fill_value=0, chunk_shape=[128,128])

In [9]:
out_ts_arr.write(grayscale_8bit).result()